## Import package

In [1]:
from bs4 import BeautifulSoup 
import re
import urllib
import requests
import shutil 
from selenium import webdriver
import os
import time
import copy
from tqdm import tqdm
from pykospacing import spacing
import pandas as pd
import copy
import string, random ## generate random str package
from collections import OrderedDict ## repetition removal package
from collections import Counter

Using TensorFlow backend.


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [2]:
def find_rgb(value):
    p = re.compile('rgb.*\)')
    rgb = p.findall(value)
    rgb = str(rgb[0])
    return rgb

In [3]:
def numeric_value(string):
    number = re.sub('[^0-9]', '', string) 
    return(number)

In [21]:
User_id = 'icej10'
Post_id = '221321717810'


url = "http://blog.naver.com/PostView.nhn?blogId="+User_id+ "&logNo=" + Post_id +"&redirect=Dlog&widgetTypeCall=true"
r = requests.get(url)
bs = BeautifulSoup(re.sub('&nbsp;',' ',r.text).encode("utf-8"), "html.parser")

In [22]:
# structure
structure = bs.find("div", {"id": "postViewArea"})
if structure == None:
    structure = bs.find("div",{"class","se_component_wrap sect_dsc __se_component_area"})

## Align

In [23]:
split_structure =  remove_odd(str(structure)).split('>')
center= 0
left = 0
right = 0
justify = 0
for item in split_structure:
    if 'align' or 'ALIGN' in str(item):
        if 'center' in str(item):
            center +=1
        if 'left' in str(item):
            left += 1
        if 'right' in str(item):
            right +=1
        if 'justify' in str(item):
            justify +=1

In [24]:
## Maximum or 0이면 default값으로 셋팅 한 것.
Align = [left, center, right, justify]
Align

[8, 48, 1, 24]

## Font Style

In [76]:
fonts = ['sans-serif','nanumbarungothic','nanumgothic','nanummyeongjo','돋움','돋움체','굴림','굴림체','바탕','바탕체','궁서','Arial','Tahoma','Times New Roman','Verdana','Couier New']

In [77]:
font_list = []
for item in split_structure:
    for font in fonts:
        if font in item:
            font_list.append(font)
cnt = Counter(font_list)
basic_font = cnt.most_common(1)
if basic_font == []:
    basic_font = 'default'

In [78]:
basic_font

'default'

## Font_size

In [79]:
def numeric_value(string):
    number = re.sub('[^0-9]', '', string) 
    return(number)

In [80]:
font_size_list = []
for item in split_structure:
    if 'font' in item:
        p = re.compile('font-size:.*pt;')
        font_size = p.findall(str(item))
        string = "".join(font_size)
        size = numeric_value(string)
        if len(size) < 3 and 0 < len(size):
            font_size_list.append(numeric_value(string))
cnt = Counter(font_size_list)
basic_font_size = cnt.most_common(1)
if basic_font_size == []:
    basic_font_size = 'default'

## B,strong_tag 

In [81]:
b_strong_tag = structure.find_all(['b','strong'])

In [82]:
## word는 무조건 remove_odd 하기!!!
b_strong_rgb_list = []
b_strong_rgb_word_list = []
b_strong_rgb_word_len = 0
for item in b_strong_tag:
    if len(item.text) > 1 and len(item.text) < 22:
        b_strong_rgb_list.append(find_rgb(str(item)))
        b_strong_rgb_word_list.append(remove_odd(item.text))
        b_strong_rgb_word_len += len(remove_odd(item.text))
b_strong_count = len(b_strong_rgb_list)

In [83]:
[b_strong_rgb_list,b_strong_rgb_word_list,b_strong_count,b_strong_rgb_word_len]

[['rgb(58, 50, 195)',
  'rgb(255, 0, 0)',
  'rgb(255, 0, 0)',
  'rgb(58, 50, 195)',
  'rgb(255, 0, 0)',
  'rgb(255, 0, 0)',
  'rgb(255, 0, 0)',
  'rgb(255, 0, 0)'],
 ['ice J의 음악이야기',
  '"독식하려 하지 말고 나누라!"',
  '답: 사용자',
  '3. 네이버 이탈인가? 검색 이탈인가?',
  '파워블로그...',
  '1. 전업 블로거를 양산해야 한다.',
  '2. 파워 블로그를 부활 시켜야 한다.',
  '3. 광고성 블로그를 정리해야 한다.'],
 8,
 124]

## Color_tag
- rgb 0, 0, 0 or rgb(000,000,000) 이상으로 뽑기

In [84]:
color_tag =  structure.find_all('span')

In [85]:
## word는 무조건 remove_odd 하기!!!
color_rgb_list = []
color_rgb_word_list = []
color_rgb_word_len = 0
for item in color_tag:
    if 'rgb' in str(item):
        if len(item.text)>1 and len(item.text) < 25:
            color_rgb_list.append(find_rgb(str(item)))
            color_rgb_word_list.append(remove_odd(item.text))
            color_rgb_word_len += len(remove_odd(item.text))
color_count = len(color_rgb_word_list)

In [86]:
[color_rgb_list,color_rgb_word_list,color_count,color_rgb_word_len]

[['rgb(58, 50, 195)',
  'rgb(255, 0, 0)',
  'rgb(255, 0, 0)',
  'rgb(58, 50, 195)',
  'rgb(58, 50, 195)',
  'rgb(58, 50, 195)',
  'rgb(255, 0, 0)',
  'rgb(255, 0, 0)',
  'rgb(255, 0, 0)',
  'rgb(255, 0, 0)'],
 ['ice J의 음악이야기',
  '"독식하려 하지 말고 나누라!"',
  '답: 사용자',
  '3. 네이버 이탈인가? 검색 이탈인가?',
  '4. 네이버 이탈의 근본 원인은 ',
  '이다.',
  '파워블로그...',
  '전업 블로거',
  '파워 블로그',
  '광고성 블로그'],
 10,
 104]

## Total_features

In [91]:
save_content_list = [Align,basic_font,basic_font_size,b_strong_rgb_list,b_strong_rgb_word_list,b_strong_count,b_strong_rgb_word_len,color_rgb_list,color_rgb_word_list,color_count,color_rgb_word_len]

In [94]:
save_content = "\t".join(list(map(str,save_content_list)))

In [95]:
with open('test_font.csv', 'a', encoding='utf-8') as f:
    f.write(save_content+"\n")